# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
!ollama pull llama3.2

In [ ]:
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"

MODEL_CHOICES = [
    ("gpt-4o-mini (OpenRouter)", MODEL_GPT, "openrouter"),
    ("llama3.2 (Ollama)", MODEL_LLAMA, "ollama"),
]

In [ ]:
load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
BASE_URL = "https://openrouter.ai/api/v1"

if api_key and api_key.startswith("sk-or-v1-") and len(api_key) > 10:
    print("OpenRouter API key looks good")
else:
    print("Check OPENROUTER_API_KEY in .env (see troubleshooting notebook if needed)")

openrouter = OpenAI(api_key=api_key, base_url=BASE_URL)
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

SYSTEM_PROMPT = """You are a professional AI coding tutor.
When given a programming question, provide a clear, step-by-step explanation that is instructive and designed for learners.
Break down concepts, highlight key ideas, use relevant code examples, and point out common mistakes.
Keep a friendly, expert tone. Respond in markdown when helpful (e.g. code blocks)."""

In [ ]:

def stream_reply(history, user_message, model_label):
    label_to_config = {label: (model_id, backend) for label, model_id, backend in MODEL_CHOICES}
    model_id, backend = label_to_config.get(model_label, MODEL_CHOICES[0][1:])

    client = openrouter if backend == "openrouter" else ollama
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for user, assistant in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant or ""})
    messages.append({"role": "user", "content": user_message})

    stream = client.chat.completions.create(
        model=model_id,
        messages=messages,
        stream=True,
    )
    for chunk in stream:
        part = chunk.choices[0].delta.content or ""
        if part:
            yield part

In [ ]:
def chat(message, history, model_choice):
    if not message or not message.strip():
        return
    full = ""
    for chunk in stream_reply(history, message, model_choice):
        full += chunk
        yield full

In [ ]:
model_dropdown = gr.Dropdown(
    choices=[label for label, _, _ in MODEL_CHOICES],
    value=MODEL_CHOICES[0][0],
    label="Model",
)

demo = gr.ChatInterface(
    chat,
    additional_inputs=[model_dropdown],
    title="Technical Q&A Tutor",
    description="Ask a coding question. Switch models with the dropdown. Replies stream in real time.",
)
demo.launch(share=True)